In [1]:
import asyncio

import sglang as sgl
# import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

import nest_asyncio

nest_asyncio.apply()

In [2]:
llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

INFO 10-04 10:30:48 __init__.py:179] Automatically detected platform rocm.
WARNING 10-04 10:30:48 rocm.py:34] `fork` method is not supported by ROCm. VLLM_WORKER_MULTIPROC_METHOD is overridden to `spawn` instead.


INFO 10-04 10:30:57 __init__.py:179] Automatically detected platform rocm.
INFO 10-04 10:30:57 __init__.py:179] Automatically detected platform rocm.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.73it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.72it/s]

Capturing batches (bs=1 avail_mem=46.71 GB): 100%|██████████| 51/51 [00:09<00:00,  5.35it/s]  


In [6]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

In [7]:
sampling_params = {
    "temperature": 0.8, 
    "top_p": 0.95,
    "max_new_tokens": 512,
}

#### Non-streaming Synchronous Generation

In [8]:
outputs = llm.generate(prompts, sampling_params)

for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")
    print()

[aiter] type hints mismatch, override to --> mha_batch_prefill(q: torch.Tensor, k: torch.Tensor, v: torch.Tensor, cu_seqlens_q: torch.Tensor, kv_indptr: torch.Tensor, kv_page_indices: torch.Tensor, max_seqlen_q: int, max_seqlen_k: int, dropout_p: float, softmax_scale: float, logits_soft_cap: float, zero_tensors: bool, is_causal: bool, window_size_left: int, window_size_right: int, return_softmax_lse: bool, return_dropout_randval: bool, out: Optional[torch.Tensor] = None, bias: Optional[torch.Tensor] = None, alibi_slopes: Optional[torch.Tensor] = None, gen: Optional[torch.Generator] = None) -> list[torch.Tensor]
[2025-10-04 10:34:41] type hints mismatch, override to --> mha_batch_prefill(q: torch.Tensor, k: torch.Tensor, v: torch.Tensor, cu_seqlens_q: torch.Tensor, kv_indptr: torch.Tensor, kv_page_indices: torch.Tensor, max_seqlen_q: int, max_seqlen_k: int, dropout_p: float, softmax_scale: float, logits_soft_cap: float, zero_tensors: bool, is_causal: bool, window_size_left: int, window_

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [insert name], and I'm an [insert age or profession]. I'm a [insert occupation] and I'm currently [insert current occupation]. I enjoy [insert hobbies, interests, or passions], and I'm always looking for ways to grow and learn new things. I believe that my unique combination of skills and experiences makes me a valuable asset to anyone I meet. What kind of character are you? [insert any additional information or details about the character that you think will make the reader feel more comfortable and familiar with your personality. ] [insert the character]
[Insert name] is a friendly, outgoing, and confident person. He has a friendly and outgoing personality, and he is always ready to lend a helping hand. He is known for his kindness, warmth, and sense of humor. He has a great sense of humor and is always smiling, which makes him a joy to be around. He is very empathetic and e

#### Streaming Synchronous Generation

In [9]:
print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print("===============================")
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [insert your name], and I am a [insert your occupation or field of interest]. I am a [insert your profession or vocation]. I am [insert your age] years old, [insert your gender], and I hail from [insert your country or region]. I am passionate about [insert your passion or interest], and I thrive on [insert your hobby or activity]. Throughout my career, I have [insert your achievements or accomplishments], and I am known for my [insert your qualities or attributes]. If you had the opportunity to have dinner with me, what would you serve me? I would serve you with my [insert your personality, interests, and hobbies]. Thank you for having me. What is your occupation or field of interest? I am a [insert your occupation or vocation]. What is your age? I am [insert your age]. What is your gender? I am [insert y

#### Non-streaming Asynchronous Generation

In [10]:
print("\n=== Testing asynchronous batch generation ===")

async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print("===============================")
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")
        print()

asyncio.run(main())


=== Testing asynchronous batch generation ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  Jane. I'm a friendly, witty, and curious AI assistant. How can I help you today? Any questions or concerns you may have? I'm always here to assist you. Thank you. What are you working on right now?
Jane, AI assistant, is always ready to assist with a wide range of questions and concerns. How can I help you today? I'm here to help you with any questions or concerns you may have, and I'm always here to assist you. How can I help you today? I'm always here to assist you with a wide range of questions and concerns. How can I help you today? I'm always here to assist you with a wide range of questions and concerns. How can I help you today? I'm always here to assist you with a wide range of questions and concerns. How can I help you today? I'm always here to assist you with a wide range of questions and concerns. How can I help you to

#### Streaming Asynchronous Generation

In [11]:
print("\n=== Testing asynchronous streaming generation (no repeats) ===")

async  def main():
    for prompt in prompts:
        print("===============================")
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)
        print()

asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Your Name], and I am a [X] with a passion for [Y]. Let me share a little bit about myself, and I'll be happy to tell you more about you and your story. What can you tell me about yourself? What's your background, education, and what do you enjoy doing? And how do you feel about sharing your story with the world? That's all I need to know to get to know you. Welcome, [Recipient's Name], and I'm excited to have you on my team. Here's to learning about you, and sharing your story with the world. Let's do this! 🚀✨ #MeetThePerson #Charististics #CharacterDevelopment #Storytelling #InterpersonalSkills #FriendshipBuilding #Networking #PersonalStory #Self-Reflection #Empathy #Cultivation #Passion #Purpose #NewPerspectives #Insight #Leadership #Humor #Interactions #HumanKind #EmotionalIntelligence #SocialNetworking #Crea